Task 2 - Understanding how long users actively work on designs?
You are part of a cross functional team who would like to improve product engagement of our modeling features. Your Product Manager would like to understand how long our users are working on a design before they export it. 
Your task is to give some insight on this from the provided dat.


In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None)

import sqlite3
from tabulate import tabulate
import matplotlib.pyplot as plt
import seaborn as sns

Action plan:
1. design_event dataset: excluding workspace_uuid <> workspace_id
2. design_event dataset: putting the workspace_uuid value where the workspace_id is NULL
3. design_event dataset: putting the workspace_id value where the workspace_uuid is NULL
4. both datasets: Join tables

In [ ]:
#design_events = pd.read_csv('/content/DA_home_task_files_design_events.csv')
design_events = pd.read_csv(r"C:\Users\lily\Desktop\DS interview prep\Interviews\Shapr3D\DA_home_task_files_design_events.csv")

In [ ]:
#design_sessions = pd.read_csv('/content/DA_homework_files_design_sessions.csv')
design_sessions = pd.read_csv(r"C:\Users\lily\Desktop\DS interview prep\Interviews\Shapr3D\DA_homework_files_design_sessions.csv")

*Colum names extraction*

In [ ]:
design_events.columns

Index(['event_name', 'device_platform', 'event_time', 'user_id',
       'active_team_id', 'app_version', 'workspace_id', 'workspace_uuid',
       'file_format', 'resolution'],
      dtype='object')

*Connect to SQLite - automatical database file creation*

In [ ]:
conn = sqlite3.connect("shapr3d.db")
cursor = conn.cursor()

*Data load into SQLite table called design_events*

In [ ]:
design_events.to_sql("temp_events", conn, if_exists="replace", index=False)

73008

*Create the SQLite table creation (inside of the database) for efficient queries and adding primary key*

In [ ]:
cursor.execute("""
    CREATE TABLE events (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        event_name TEXT,
        device_platform TEXT,
        event_time TEXT,
        user_id TEXT,
        active_team_id TEXT,
        app_version TEXT,
        workspace_id TEXT,
        workspace_uuid TEXT,
        file_format TEXT,
        resolution TEXT
    )
""")

cursor.execute("""
    INSERT INTO events (event_name, device_platform, event_time, user_id, active_team_id, app_version, workspace_id, workspace_uuid, file_format, resolution)
    SELECT event_name, device_platform, event_time, user_id, active_team_id, app_version, workspace_id, workspace_uuid, file_format, resolution FROM temp_events
""")

cursor.execute("DROP TABLE temp_events")
conn.commit()

In [ ]:
cursor.execute("SELECT * FROM events LIMIT 2")
rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 'design_created', 'Windows', '2024-11-26 07:55:47.896000', '84d9a7e6-e84e-4498-b963-ab37f2422054', 'd34a926b-76c3-4f25-9fd7-c1bfdb45aebc', '5.650.7126.0', None, 'fdb471ff-6a91-49aa-8d33-e9cf129ad978', None, None)
(2, 'design_created', 'Windows', '2024-10-23 14:54:54.831000', '47405968-d781-4fa1-8f63-85e54b58a7bc', 'd56aa67d-6879-46e7-bb20-3a086fcb0050', '5.730.7613.0', None, '2b586791-b98e-4ba7-8c27-c2ac1cb28918', None, None)


*Deleting duplications*

In [ ]:
cursor.execute('''
WITH Duplicates AS (
    SELECT MIN(ROWID) AS keep_rowid
    FROM events
    GROUP BY event_name, device_platform, event_time, user_id, active_team_id, app_version, workspace_id, workspace_uuid, file_format, resolution
    HAVING COUNT(*) > 1
)

DELETE FROM events
WHERE ROWID NOT IN (
    SELECT MIN(ROWID)
    FROM events
    GROUP BY event_name, device_platform, event_time, user_id, active_team_id, app_version, workspace_id, workspace_uuid, file_format, resolution
)
''')

conn.commit()

**Excluding workspace_uuid <> workspace_id**

In [ ]:
cursor.execute('''
SELECT different, COUNT(*)
FROM (
    SELECT *,
        CASE
            WHEN workspace_id IS NOT NULL AND workspace_uuid IS NOT NULL AND workspace_id <> workspace_uuid THEN 'Yes'
            WHEN workspace_id IS NOT NULL AND workspace_uuid IS NOT NULL THEN 'No'
            ELSE 'NA'
        END AS different
    FROM events
)
GROUP BY different;
''')

diff  = cursor.fetchall()
diff

[('NA', 61573), ('No', 11374), ('Yes', 17)]

In [ ]:
cursor.execute('''
    SELECT event_name, event_time, user_id, workspace_id, workspace_uuid
    FROM events
    WHERE workspace_id IS NOT NULL
      AND workspace_uuid IS NOT NULL
      AND workspace_id <> workspace_uuid;
''')


diff02 = cursor.fetchall()
headers = ["Event name", "Event time", "User ID", "Workspace ID", "Workspace UUID"]

print(tabulate(diff02, headers=headers, tablefmt="grid"))

+---------------------+----------------------------+--------------------------------------+--------------------------------------+--------------------------------------+
| Event name          | Event time                 | User ID                              | Workspace ID                         | Workspace UUID                       |
+=====================+============================+======================================+======================================+======================================+
| design_open_tapped  | 2024-11-15 20:33:01.495000 | 512c9512-25d7-49b2-9665-6398b0fecd04 | 62e32c24-83ba-484b-a0da-25861f5e9be3 | 1398afc4-4efa-4776-ae8b-d3cbf60501a2 |
+---------------------+----------------------------+--------------------------------------+--------------------------------------+--------------------------------------+
| design_open_tapped  | 2025-01-20 18:58:20.839000 | 512c9512-25d7-49b2-9665-6398b0fecd04 | 7f383594-7d0d-4fb4-8a8c-25da74624552 | 92682a03-cfcf-4a26-

In [ ]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS events_copy AS
    SELECT * FROM events;
''')

conn.commit()

print("A copy of the 'events' table has been created as 'events_copy'.")

A copy of the 'events' table has been created as 'events_copy'.


In [ ]:
cursor.execute('''
    DELETE FROM events
    WHERE workspace_id IS NOT NULL
      AND workspace_uuid IS NOT NULL
      AND workspace_id <> workspace_uuid;
''')

conn.commit()

In [ ]:
cursor.execute('''
    SELECT COUNT(*)
    FROM events
''')

rows02 = cursor.fetchone()[0]
rows02

72947

**Workspace_uuid where workspace_id is NULL**

In [ ]:
cursor.execute('''
    UPDATE events
    SET workspace_id = workspace_uuid
    WHERE workspace_id IS NULL AND workspace_uuid IS NOT NULL;
''')

conn.commit()

In [ ]:
cursor.execute('''
    SELECT COUNT(*)
    FROM events
    WHERE workspace_id IS NULL
    LIMIT 2
''')

wpidnull = cursor.fetchall()
wpidnull

[(0,)]

**Workspace_id where workspace_uuid is NULL**

In [ ]:
cursor.execute('''
    UPDATE events
    SET workspace_uuid = workspace_id
    WHERE workspace_uuid IS NULL AND workspace_id IS NOT NULL;
''')

conn.commit()

In [ ]:
cursor.execute('''
    SELECT COUNT(*)
    FROM events
    WHERE workspace_uuid IS NULL
    LIMIT 2
''')

wpuuidnull = cursor.fetchall()
wpuuidnull

[(0,)]

**workspace_uuid - workspace_iid check**

In [ ]:
cursor.execute('''
SELECT different, COUNT(*)
FROM (
    SELECT *,
        CASE
            WHEN workspace_id IS NOT NULL AND workspace_uuid IS NOT NULL AND workspace_id <> workspace_uuid THEN 'Yes'
            WHEN workspace_id IS NOT NULL AND workspace_uuid IS NOT NULL THEN 'No'
            ELSE 'NA'
        END AS different
    FROM events
)
GROUP BY different;
''')

diff03  = cursor.fetchall()
diff03

[('No', 72947)]

*Column names extraction*

In [ ]:
design_sessions.columns

Index(['user_id', 'workspace_id', 'platform', 'session_start', 'session_end',
       'session_length_secs', 'session_length_mins'],
      dtype='object')

*Data load into SQLite table called design_sessions*

In [ ]:
design_sessions.to_sql("temp_events", conn, if_exists="replace", index=False)

48747

*Create the SQLite table creation (inside of the database) for efficient queries and adding primary key*

In [ ]:
cursor.execute("""
    CREATE TABLE sessions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id TEXT,
        workspace_id TEXT,
        platform TEXT,
        session_start TEXT,
        session_end TEXT,
        session_length_secs INT,
        session_length_mins INT
    )
""")

cursor.execute("""
    INSERT INTO sessions (user_id, workspace_id, platform, session_start, session_end, session_length_secs, session_length_mins)
    SELECT user_id, workspace_id, platform, session_start, session_end, session_length_secs, session_length_mins FROM temp_events
""")

cursor.execute("DROP TABLE temp_events")
conn.commit()

In [ ]:
cursor.execute("SELECT * FROM sessions LIMIT 2")
rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 'f24b3056-250c-4478-94c7-5da98207585e', '9a5b0aad-62ed-4f4d-b4eb-c099f7799060', 'iOS', '2024-10-12 18:04:54.033000', '2024-10-12 18:36:44.853000', 1910, 32)
(2, '55439986-d893-4fb2-a37e-a6b814c988a6', '123db975-bb40-4052-8fa3-365614853a25', 'Windows', '2024-09-09 13:01:52.167000', '2024-09-09 13:01:52.302000', 0, 0)


**Checking for duplicates**

In [ ]:
cursor.execute('''
SELECT COUNT(*)
FROM (
    SELECT COUNT(*)
    FROM sessions
    GROUP BY user_id, workspace_id, platform, session_start, session_end, session_length_secs, session_length_mins
    HAVING COUNT(*) > 1
) AS duplicates
''')

duplicates_session = cursor.fetchone()[0]
duplicates_session

0

No duplicates in the session dataset

**Checking for missing values**

In [ ]:
query = '''
SELECT
    SUM(CASE WHEN user_id IS NULL THEN 1 ELSE 0 END) AS missing_event_name,
    SUM(CASE WHEN workspace_id IS NULL THEN 1 ELSE 0 END) AS missing_device_platform,
    SUM(CASE WHEN platform IS NULL THEN 1 ELSE 0 END) AS missing_event_time,
    SUM(CASE WHEN session_start IS NULL THEN 1 ELSE 0 END) AS missing_user_id,
    SUM(CASE WHEN session_end IS NULL THEN 1 ELSE 0 END) AS missing_active_team_id,
    SUM(CASE WHEN session_length_secs IS NULL THEN 1 ELSE 0 END) AS missing_app_version,
    SUM(CASE WHEN session_length_mins IS NULL THEN 1 ELSE 0 END) AS missing_workspace_id
FROM sessions
'''

cursor.execute(query)
missing_values_sessions = cursor.fetchall()

columns = [
    'user_id', 'workspace_id', 'platform', 'session_start', 'session_end', 'session_length_secs', 'session_length_mins'
]

missing_sessions = pd.DataFrame(missing_values_sessions, columns=columns)

print("\nMissing values in each column:")
missing_sessions


Missing values in each column:


,user_id,workspace_id,platform,session_start,session_end,session_length_secs,session_length_mins
0,0,0,0,0,0,0,0


**Null values in session_length_mins**

In [ ]:
cursor.execute('''
SELECT COUNT(DISTINCT workspace_id) AS num_workspaces_with_zero_mins
FROM sessions
WHERE session_length_mins = 0;
''')

null_mins = cursor.fetchall()
null_mins

headers = ["Num_workspaces_with_zero_mins"]

print(tabulate(null_mins, headers=headers, tablefmt="grid"))

+---------------------------------+
|   Num_workspaces_with_zero_mins |
+=================================+
|                            5998 |
+---------------------------------+


No missing values in session dataset

**Same workspace hasin zero value for mins - but also having non-zeros too**

In [ ]:
cursor.execute('''
SELECT COUNT(*)
FROM (
    SELECT workspace_id
    FROM sessions
    WHERE session_length_mins = 0
    INTERSECT
    SELECT workspace_id
    FROM sessions
    WHERE session_length_mins > 0
) AS mixed_workspaces;
''')

both_zero_nonzero = cursor.fetchall()
both_zero_nonzero

headers = ["Both_zero_nonzero "]

print(tabulate(both_zero_nonzero , headers=headers, tablefmt="grid"))

+----------------------+
|   Both_zero_nonzero  |
+======================+
|                 4107 |
+----------------------+


In [ ]:
cursor.execute('''
SELECT COUNT(*)
FROM (
    SELECT workspace_id
    FROM sessions
    GROUP BY workspace_id
    HAVING MAX(session_length_mins) = 0
) AS all_zero_workspaces;
''')

single_zero_workspaces = cursor.fetchall()
single_zero_workspaces

headers = ["Single_zero_workspaces"]

print(tabulate(single_zero_workspaces, headers=headers, tablefmt="grid"))

+--------------------------+
|   Single_zero_workspaces |
+==========================+
|                     1891 |
+--------------------------+


In [ ]:
4107 + 1891

5998

**Minutes spent on number of sessions**

In [ ]:
cursor.execute('''
SELECT session_length_mins, COUNT(workspace_id) AS workspace_count
FROM sessions
GROUP BY session_length_mins
ORDER BY workspace_count DESC
LIMIT 15;
''')

minutes_sessions = cursor.fetchall()
minutes_sessions

headers = ["Minutes", "Number of workspace_count"]

print(tabulate(minutes_sessions, headers=headers, tablefmt="grid"))

+-----------+-----------------------------+
|   Minutes |   Number of workspace_count |
+===========+=============================+
|         0 |                       10153 |
+-----------+-----------------------------+
|         1 |                        4706 |
+-----------+-----------------------------+
|         2 |                        3228 |
+-----------+-----------------------------+
|         3 |                        2678 |
+-----------+-----------------------------+
|         4 |                        2281 |
+-----------+-----------------------------+
|         5 |                        2035 |
+-----------+-----------------------------+
|         6 |                        1836 |
+-----------+-----------------------------+
|         7 |                        1638 |
+-----------+-----------------------------+
|         8 |                        1472 |
+-----------+-----------------------------+
|         9 |                        1291 |
+-----------+-------------------

**Calculating hours and adding as new column**

In [ ]:
cursor.execute('ALTER TABLE sessions ADD COLUMN session_length_hours REAL')

cursor.execute('''
UPDATE sessions
SET session_length_hours = session_length_mins / 60;
''')

conn.commit()

In [ ]:
cursor.execute('''
SELECT *
FROM sessions
WHERE session_length_mins > 60
AND session_length_hours IS NULL
LIMIT 3
''')

added_hour_check = cursor.fetchall()
added_hour_check

[]

**Joining the events and the session tables**

In [ ]:
cursor.execute('''
SELECT COUNT(DISTINCT s.workspace_id) AS unmatched_workspace_count
FROM sessions s
LEFT JOIN events e
    ON s.workspace_id = e.workspace_id
WHERE e.workspace_id IS NULL;
''')

unmatched_workspace_count = cursor.fetchall()
unmatched_workspace_count

headers = ["Unmatched_workspace_count"]

print(tabulate(unmatched_workspace_count, headers=headers, tablefmt="grid"))

+-----------------------------+
|   Unmatched_workspace_count |
+=============================+
|                           0 |
+-----------------------------+


In [ ]:
cursor.execute('''
SELECT COUNT(DISTINCT s.workspace_id) AS common_workspace_count
FROM sessions s
JOIN events e
    ON s.workspace_id = e.workspace_id;
''')

common_workspace_count = cursor.fetchall()
common_workspace_count

headers = ["Common_workspace_count"]

print(tabulate(common_workspace_count, headers=headers, tablefmt="grid"))

+--------------------------+
|   Common_workspace_count |
+==========================+
|                    14884 |
+--------------------------+


All the workspace_id are in both of the tables.

**Top15 longest time spent on workspace**

In [ ]:
cursor.execute('''
SELECT
    e.workspace_id,
    COUNT(s.session_start) AS total_sessions,
    SUM(s.session_length_mins) AS total_session_length_mins,
    SUM(s.session_length_hours) AS total_session_length_hours
FROM events e
JOIN sessions s
    ON e.workspace_id = s.workspace_id
GROUP BY e.workspace_id
ORDER BY total_session_length_hours DESC
LIMIT 8
''')

wpid_sessionlength_top15 = cursor.fetchall()
wpid_sessionlength_top15

headers = ["Workspace_id", "Total_sessions","Total_session_length_mins", "Total_session_length_hours"]

print(tabulate(wpid_sessionlength_top15, headers=headers, tablefmt="grid"))

+--------------------------------------+------------------+-----------------------------+------------------------------+
| Workspace_id                         |   Total_sessions |   Total_session_length_mins |   Total_session_length_hours |
+======================================+==================+=============================+==============================+
| 62e32c24-83ba-484b-a0da-25861f5e9be3 |            53120 |                      700160 |                         3200 |
+--------------------------------------+------------------+-----------------------------+------------------------------+
| e05fe368-3533-4a84-bb32-fec24d8a50fb |            18900 |                      283320 |                         1260 |
+--------------------------------------+------------------+-----------------------------+------------------------------+
| 5ac0b8c1-6c83-405e-b6bd-c4c1e14bbe5f |            52334 |                      558866 |                         1146 |
+-------------------------------

In [ ]:
cursor.execute('''
SELECT
    e.workspace_id,
    COUNT(s.session_start) AS total_sessions,
    SUM(s.session_length_mins) AS total_session_length_mins,
    SUM(s.session_length_hours) AS total_session_length_hours
FROM events e
JOIN sessions s
    ON e.workspace_id = s.workspace_id
GROUP BY e.workspace_id
ORDER BY total_sessions DESC
LIMIT 5
''')

total_sessions = cursor.fetchall()
total_sessions

headers = ["Workspace_id", "Total_sessions","Total_session_length_mins", "Total_session_length_hours"]

print(tabulate(total_sessions, headers=headers, tablefmt="grid"))

+--------------------------------------+------------------+-----------------------------+------------------------------+
| Workspace_id                         |   Total_sessions |   Total_session_length_mins |   Total_session_length_hours |
+======================================+==================+=============================+==============================+
| 62e32c24-83ba-484b-a0da-25861f5e9be3 |            53120 |                      700160 |                         3200 |
+--------------------------------------+------------------+-----------------------------+------------------------------+
| 5ac0b8c1-6c83-405e-b6bd-c4c1e14bbe5f |            52334 |                      558866 |                         1146 |
+--------------------------------------+------------------+-----------------------------+------------------------------+
| fe147c57-9525-466d-94db-0e267e7bb5bf |            21684 |                      129636 |                            0 |
+-------------------------------

In [ ]:
cursor.execute('''
SELECT
    e.workspace_id,
    COUNT(s.session_start) AS total_sessions,
    SUM(s.session_length_hours) AS total_session_length_hours
FROM events e
JOIN sessions s
    ON e.workspace_id = s.workspace_id
GROUP BY e.workspace_id
ORDER BY total_session_length_hours ASC
LIMIT 7
''')

wpid_sessionlength_last15 = cursor.fetchall()
wpid_sessionlength_last15

headers = ["Workspace_id", "Total_sessions", "Total_session_length_hours"]

print(tabulate(wpid_sessionlength_last15, headers=headers, tablefmt="grid"))

+--------------------------------------+------------------+------------------------------+
| Workspace_id                         |   Total_sessions |   Total_session_length_hours |
+======================================+==================+==============================+
| 001c09e8-5b7a-432d-af5a-2e52ecbf5848 |              468 |                            0 |
+--------------------------------------+------------------+------------------------------+
| 0020129b-116b-4310-873e-b8a36940888d |                2 |                            0 |
+--------------------------------------+------------------+------------------------------+
| 00233201-88f2-4756-87c1-3966e584c7b7 |                1 |                            0 |
+--------------------------------------+------------------+------------------------------+
| 00295501-9c2e-4403-bcd6-4b1d1320626a |                6 |                            0 |
+--------------------------------------+------------------+------------------------------+

**Copy & deleting the session_length_minutes with zero values**

In [ ]:
cursor.execute('''
CREATE TABLE sessions_backup AS
SELECT * FROM sessions;
''')

conn.commit()


In [ ]:
cursor.execute('''
DELETE FROM sessions
WHERE session_length_mins = 0;
''')

conn.commit()

In [ ]:
cursor.execute('''
PRAGMA table_info(sessions);
''')

columns = cursor.fetchall()
columns

[(0, 'id', 'INTEGER', 0, None, 1),
 (1, 'user_id', 'TEXT', 0, None, 0),
 (2, 'workspace_id', 'TEXT', 0, None, 0),
 (3, 'platform', 'TEXT', 0, None, 0),
 (4, 'session_start', 'TEXT', 0, None, 0),
 (5, 'session_end', 'TEXT', 0, None, 0),
 (6, 'session_length_secs', 'INT', 0, None, 0),
 (7, 'session_length_mins', 'INT', 0, None, 0),
 (8, 'session_length_hours', 'REAL', 0, None, 0)]

In [ ]:
cursor.execute('''
SELECT session_length_mins,session_length_hours
FROM sessions
LIMIT 3
''')

zero_deleted = cursor.fetchall()
zero_deleted

headers = ["Total_session_length_mins", "Total_session_length_hours"]

print(tabulate(zero_deleted, headers=headers, tablefmt="grid"))


+-----------------------------+------------------------------+
|   Total_session_length_mins |   Total_session_length_hours |
+=============================+==============================+
|                          32 |                            0 |
+-----------------------------+------------------------------+
|                           6 |                            0 |
+-----------------------------+------------------------------+
|                          11 |                            0 |
+-----------------------------+------------------------------+


**Checeking the last 15 -after zero values deleted**

In [ ]:
cursor.execute('''
SELECT
    e.workspace_id,
    COUNT(s.session_start) AS total_sessions,
    SUM(s.session_length_mins) AS total_session_length_mins,
    SUM(s.session_length_mins) / 60.0 AS total_session_length_hours
FROM events e
JOIN sessions s
    ON e.workspace_id = s.workspace_id
GROUP BY e.workspace_id
ORDER BY total_session_length_hours ASC
LIMIT 7
''')

wpid_sessionlength_last15_02 = cursor.fetchall()
wpid_sessionlength_last15_02

headers = ["Workspace_id", "Total_sessions", "Total_session_length_mins", "Total_session_length_hours"]

print(tabulate(wpid_sessionlength_last15_02, headers=headers, tablefmt="grid"))

+--------------------------------------+------------------+-----------------------------+------------------------------+
| Workspace_id                         |   Total_sessions |   Total_session_length_mins |   Total_session_length_hours |
+======================================+==================+=============================+==============================+
| 0066bbbf-5000-4e58-90d8-0f87a7d9264b |                1 |                           1 |                    0.0166667 |
+--------------------------------------+------------------+-----------------------------+------------------------------+
| 00a8afcd-a8e2-4524-aa68-4a4ae0a92984 |                1 |                           1 |                    0.0166667 |
+--------------------------------------+------------------+-----------------------------+------------------------------+
| 01918d34-3cd0-4f60-af33-b9f26613f282 |                1 |                           1 |                    0.0166667 |
+-------------------------------

Since there are many 1,2,3 minutes sessions, even though I deleted the zero values, this could also distort our average time spent on workspaces.

In [ ]:
conn.close()